In [ ]:
import keras
from keras import backend as K
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [ ]:
model = Sequential([Dense(16,input_shape=(1,),activation='relu'),
                   Dense(32,activation='relu'),
                   Dense(2,activation='softmax')])

In [ ]:
model.compile(Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(training_samples, traning_labels,validation_split=0.2,batch_size=10,epochs=20,shuffle=True,verbose=2)

In [ ]:
predictions = model.predict(scaled_test_samples,batch_size=10,verbose=0)

In [ ]:
pd.DataFrame(ppc['bus'])

In [ ]:
from pypower.api import case9, ppoption, runpf, printpf, makeB
from scipy.sparse import dok_matrix
from pypower.ppoption import ppoption
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
ppc = case9()

bus = pd.DataFrame(ppc['bus'])
bus[0] = bus[0] - 1
ppc['bus'] = bus.to_numpy()

branch = pd.DataFrame(ppc['branch'])
branch[0] = branch[0] - 1
branch[1] = branch[1] - 1
ppc['branch'] = branch.to_numpy()

In [ ]:
pd.DataFrame(ppc['gen'])

In [ ]:
org_branch = ppc['branch']
org_bus = ppc['bus']
data = {}

In [ ]:
def masti(a,b):
    for i in range(a,b):
        bus = pd.DataFrame(org_bus) 
        bus.loc[4,2] = (bus.loc[4,2])*(np.random.uniform(0.8,1.2))
        bus.loc[6,2] = (bus.loc[6,2])*(np.random.uniform(0.8,1.2))
        bus.loc[8,2] = (bus.loc[8,2])*(np.random.uniform(0.8,1.2))
        bus.loc[4,3] = (bus.loc[4,3])*(np.random.uniform(0.8,1.2))*(np.random.uniform(0.15,0.25))
        bus.loc[6,3] = (bus.loc[6,3])*(np.random.uniform(0.8,1.2))*(np.random.uniform(0.15,0.25))
        bus.loc[8,3] = (bus.loc[8,3])*(np.random.uniform(0.8,1.2))*(np.random.uniform(0.15,0.25))
        bus[2] = bus[2]*-1
        ppc['bus'] = np.array(bus)
        ppc['branch'] = org_branch
        row = np.random.randint(0,ppc['branch'].shape[0])
        ppc['branch'] = np.delete(ppc['branch'],row,0)
        ppopt = ppoption(PF_ALG=1)
        try:
            r = runpf(ppc, ppopt)
            data[f'sim{i}'] = r
            #lst_suc.append(((bus.loc[4,2],bus.loc[6,2],bus.loc[8,2]),(bus.loc[4,3],bus.loc[6,3],bus.loc[8,3])))
        except:
            #lst_fail.append(((bus.loc[4,2],bus.loc[6,2],bus.loc[8,2]),(bus.loc[4,3],bus.loc[6,3],bus.loc[8,3])))
            continue

In [ ]:
org_bus

In [ ]:
import multiprocessing

for i in [[(1,10001),(10001,20001),(30001,40001),(40001,50001)]]:
    
    if __name__ == "__main__":
        
        p1 = multiprocessing.Process(target=masti(i[0][0],i[0][1]))
        p2 = multiprocessing.Process(target=masti(i[1][0],i[1][1]))
        p3 = multiprocessing.Process(target=masti(i[2][0],i[2][1]))
        p4 = multiprocessing.Process(target=masti(i[3][0],i[3][1]))
        #p5 = multiprocessing.Process(target=masti(i[4][0],i[4][1]))
    
 
        p1.start()
        p2.start()
        p3.start()
        p4.start()
        #p5.start()
   
        p1.join()
        p2.join()
        p3.join()
        p4.join()
        #p5.join()

In [ ]:
dupli = data.copy()

In [ ]:
for i in dupli:
    print(i)

In [ ]:
bus_voltages = {}
for i in data:
    bus_voltages[f"{i}"] = pd.DataFrame(data[i][0]['order']['int']['bus'])[7]

In [ ]:
print(('data:',len(data)),('bus_voltages:',len(bus_voltages)))

In [ ]:
limits = {'n':1,'desired_voltage':1,'alarm_low':0.93, 'alarm_up':1.07,'sec_low':0.90,'sec_up':1.10}

In [ ]:
sec_ind = {}
volsec_label = {}
avoid = []
#sum_1 = 0
#sum_2 = 0
g_up = (limits['sec_up']-limits['alarm_up'])/limits['desired_voltage']
g_low = (limits['alarm_low']-limits['sec_low'])/limits['desired_voltage']
#print('g_up:',g_up)
#print('g_low:',g_low)
 
#count = 0
for i in bus_voltages:
    #print(i)
    sum_1 = 0
    sum_2 = 0
    for j in bus_voltages[i]:
        #count += 1
        #print(f"\n\n no:{count} \n\n")
        if j > limits['alarm_up']:
            #print(f"{i} is greater than upper alarm limit {limits['alarm_up']}")
            d_up = (j - limits['alarm_up'])/limits['desired_voltage']
            d_low = 0
            #print('d_up:',d_up)
            #print('d_low:',d_low)
        elif j < limits['alarm_low']:
            #print(f"{i} is lower than lower alarm limit {limits['alarm_low']}")
            d_low = (limits['alarm_low'] - j)/limits['desired_voltage']
            d_up = 0
            #print('d_up:',d_up)
            #print('d_low:',d_low)
        else:
            #print('amra nirapod re maola')
            d_up = 0
            d_low = 0
            #print('d_up:',d_up)
            #print('d_low:',d_low)
        sum_1 += (d_up/g_up)**(2*limits['n']) 
        sum_2 += (d_low/g_low)**(2*limits['n'])
    #print('sum 1:',sum_1)
    #print('sum 2:',sum_2)
    
    sum = (sum_1 + sum_2)**(1/(2*limits['n']))
    #sum = sum / 100
    #sec_ind[i] = sum
    if 0<= sum <= 2:
        sec_ind[i] = sum
    else:
        avoid.append(i)


#count = 0   
for i in sec_ind:
    if sec_ind[i] == 0:
        volsec_label[f"{i}"] = 0
        #print("index:",sum)
        #print("label:",volsec_label[f"{i}"])
        
    elif 0 < sec_ind[i] <= 1:
        #count += 1
        #if count <= 6000:
        volsec_label[f"{i}"] = 1
        #else:
            #avoid.append(i)
        #print("index:",sum)
        #print("label:",volsec_label[f"{i}"])
    
    else:
        volsec_label[f"{i}"] = 2
        #print("index:",sum)
        #print("label:",volsec_label[f"{i}"])

#for i in avoid:
    #del data[i]



In [ ]:
print(('data:',len(data)),('bus_voltages:',len(bus_voltages)))

In [ ]:
len(sec_ind)

In [ ]:
sum_0 = 0
sum_1 = 0
sum_2 = 0
for i in volsec_label:
    if volsec_label[i]==0:
        sum_0 += 1
    elif volsec_label[i] == 1:
        sum_1 += 1
    else:
        sum_2 += 1
        
print(sum_0,sum_1,sum_2)  


In [ ]:
for i in avoid:
    del data[i]

In [ ]:
len(data)

In [ ]:
inp_bp = {}
inp_bpp = {}

for i in data:
    baseMVA = data[i][0]['baseMVA']
    bus = data[i][0]['bus']
    branch = data[i][0]['branch']
    bp,bpp = makeB(baseMVA,bus,branch,2) #the method '2' needs to be checked for what it is. Can other methods be used?
    bp = np.diag(bp.toarray())
    bpp = np.diag(bpp.toarray())
    bus = pd.DataFrame(bus)
    inp_bp["{}".format(i)] = np.array([np.array(bus[2]),np.array(bus[3]),bp])
    inp_bpp["{}".format(i)] = np.array([np.array(bus[2]),np.array(bus[3]),bpp])
    
    #bp and bpp are the same ! Check.

In [ ]:
feed_bp = np.array(list(inp_bp.values()))
label = np.array(list(volsec_label.values()))


#traing samples 70% testing sample 30% of total simulations created
x_train,x_test = np.split(feed_bp,[int(len(label)*0.8)])
y_train,y_test = np.split(label,[int(len(label)*0.8)])


x_train = x_train.reshape(len(x_train),x_train[0].shape[0],x_train[0].shape[1],1)
x_test = x_test.reshape(len(x_test),x_test[0].shape[0],x_test[0].shape[1],1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
len(data)*0.7

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,Reshape, MaxPooling2D, Dropout
from keras import regularizers

In [ ]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(12, kernel_size=(3,3), padding ='same',activation='relu',input_shape = (3,9,1)))
model.add(Conv2D(24, kernel_size=(3,3),padding ='same', activation='relu'))
model.add(Conv2D(48, kernel_size=(3,3),padding ='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3),padding ='same', activation='relu',kernel_regularizer = regularizers.l2(0.01)))
#model.add(Conv2D(64, kernel_size=(3,3),padding ='same', activation='relu',kernel_regularizer = regularizers.l2(0.01)))

#model.add(Reshape((model.get_layer('layer_4').output_shape[3],-1)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.get_layer('layer_4').output_shape[0]

In [ ]:
1728/2/2/2/2/2

In [ ]:
model.summary()

In [ ]:
1728*9

In [ ]:
from keras.optimizers import *
model.compile(SGD(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.utils import compute_class_weight
classWeight = compute_class_weight('balanced', outputLabels, outputs) 
classWeight = dict(enumerate(classWeight))
#model.fit(X_train, y_train, batch_size = batch_size, nb_epoch = nb_epochs, show_accuracy = True, verbose = 2, validation_data = (X_test, y_test), class_weight=classWeight)

In [ ]:
model.fit(x_train,y_train,validation_split=0.3,batch_size=100,epochs=40,shuffle = True,verbose=2)

In [ ]:
test = model.predict(x_test)

In [ ]:
for i in range(test.shape[0]):
    test[i][0] = round(test[i][0])
    test[i][1] = round(test[i][1])
    test[i][2] = round(test[i][2])

In [ ]:
if np.array_equal(test[0],y_test[0]):
    print('yeah')

In [ ]:
shonkha = 0
for i in range(test.shape[0]):
    if np.array_equal(test[i],y_test[i]):
        shonkha+=1
        
print(shonkha/test.shape[0])
    

In [ ]:
shonkha

In [ ]:
rand = Sequential()
rand.add(Dense(16,input_shape=(3,9),activation='relu'))

In [ ]:
rand.summary()

In [ ]:
test

In [ ]:
new = {}
for i in range(1,101):
    bus = pd.DataFrame(org_bus) 
    bus.loc[4,2] = (bus.loc[4,2])*(np.random.uniform(0.8,1.2))
    bus.loc[6,2] = (bus.loc[6,2])*(np.random.uniform(0.8,1.2))
    bus.loc[8,2] = (bus.loc[8,2])*(np.random.uniform(0.8,1.2))
    bus.loc[4,3] = (bus.loc[4,3])*(np.random.uniform(0.8,1.2))*(np.random.uniform(0.15,0.25))
    bus.loc[6,3] = (bus.loc[6,3])*(np.random.uniform(0.8,1.2))*(np.random.uniform(0.15,0.25))
    bus.loc[8,3] = (bus.loc[8,3])*(np.random.uniform(0.8,1.2))*(np.random.uniform(0.15,0.25))
    bus[2] = bus[2]*-1
    ppc['bus'] = np.array(bus)
    ppc['branch'] = org_branch
    row = np.random.randint(0,ppc['branch'].shape[0])
    ppc['branch'] = np.delete(ppc['branch'],row,0)
    ppopt = ppoption(PF_ALG=1)
    try:
        r = runpf(ppc, ppopt)
        new[f'sim{i}'] = r
            #lst_suc.append(((bus.loc[4,2],bus.loc[6,2],bus.loc[8,2]),(bus.loc[4,3],bus.loc[6,3],bus.loc[8,3])))
    except:
            #lst_fail.append(((bus.loc[4,2],bus.loc[6,2],bus.loc[8,2]),(bus.loc[4,3],bus.loc[6,3],bus.loc[8,3])))
        continue

In [ ]:
new_voltages = {}
for i in new:
    new_voltages[f"{i}"] = pd.DataFrame(new[i][0]['order']['int']['bus'])[7]

In [ ]:
len(new_voltages)

In [ ]:
new_ind = {}
new_label = {}
new_avoid = []
#sum_1 = 0
#sum_2 = 0
g_up_new = (limits['sec_up']-limits['alarm_up'])/limits['desired_voltage']
g_low_new = (limits['alarm_low']-limits['sec_low'])/limits['desired_voltage']
#print('g_up:',g_up)
#print('g_low:',g_low)
 
#count = 0
for i in new_voltages:
    #print(i)
    sum_1 = 0
    sum_2 = 0
    for j in new_voltages[i]:
        #count += 1
        #print(f"\n\n no:{count} \n\n")
        if j > limits['alarm_up']:
            #print(f"{i} is greater than upper alarm limit {limits['alarm_up']}")
            d_up_new = (j - limits['alarm_up'])/limits['desired_voltage']
            d_low_new = 0
            #print('d_up:',d_up)
            #print('d_low:',d_low)
        elif j < limits['alarm_low']:
            #print(f"{i} is lower than lower alarm limit {limits['alarm_low']}")
            d_low_new = (limits['alarm_low'] - j)/limits['desired_voltage']
            d_up_new = 0
            #print('d_up:',d_up)
            #print('d_low:',d_low)
        else:
            #print('amra nirapod re maola')
            d_up_new = 0
            d_low_new = 0
            #print('d_up:',d_up)
            #print('d_low:',d_low)
        sum_1 += (d_up_new/g_up_new)**(2*limits['n']) 
        sum_2 += (d_low_new/g_low_new)**(2*limits['n'])
    #print('sum 1:',sum_1)
    #print('sum 2:',sum_2)
    
    sum = (sum_1 + sum_2)**(1/(2*limits['n']))
    #sum = sum / 100
    #new_ind[i] = sum

    if 0<= sum <= 2:
        new_ind[i] = sum
    else:
        new_avoid.append(i)



#count = 0   
for i in new_ind:
    if new_ind[i] == 0:
        new_label[f"{i}"] = 0
        #print("index:",sum)
        #print("label:",volsec_label[f"{i}"])
        
    elif 0 < new_ind[i] <= 1:
        #count += 1
        #if count <= 6000:
        new_label[f"{i}"] = 1
        #else:
            #avoid.append(i)
        #print("index:",sum)
        #print("label:",volsec_label[f"{i}"])
    
    else:
        new_label[f"{i}"] = 2
        #print("index:",sum)
        #print("label:",volsec_label[f"{i}"])

#for i in avoid:
    #del data[i]

In [ ]:
sum_0 = 0
sum_1 = 0
sum_2 = 0
for i in new_label:
    if new_label[i]==0:
        sum_0 += 1
    elif new_label[i] == 1:
        sum_1 += 1
    else:
        sum_2 += 1
        
print(sum_0,sum_1,sum_2)

In [ ]:
for i in new_avoid:
    del new[i]

In [ ]:
len(new)

In [ ]:
for i in new:
    print(i)

In [ ]:
del new_label['sim1']

In [ ]:
new_bp = {}
new_bpp = {}

for i in new:
    baseMVA = new[i][0]['baseMVA']
    bus = new[i][0]['bus']
    branch = new[i][0]['branch']
    bp,bpp = makeB(baseMVA,bus,branch,2) #the method '2' needs to be checked for what it is. Can other methods be used?
    bp = np.diag(bp.toarray())
    bpp = np.diag(bpp.toarray())
    bus = pd.DataFrame(bus)
    new_bp["{}".format(i)] = np.array([np.array(bus[2]),np.array(bus[3]),bp])
    new_bpp["{}".format(i)] = np.array([np.array(bus[2]),np.array(bus[3]),bpp])
    
    #bp and bpp are the same ! Check.

In [ ]:
feed_bp_new = np.array(list(new_bp.values()))
label_new = np.array(list(new_label.values()))



new_xtest = feed_bp_new
new_ytest = label_new


new_xtest = new_xtest.reshape(len(new_xtest),new_xtest[0].shape[0],new_xtest[0].shape[1],1)
new_ytest = to_categorical(new_ytest)

In [ ]:
new_xtest.shape

In [ ]:
new_ytest.shape

In [ ]:
wtv = model.predict(new_xtest)

In [ ]:
for i in range(wtv.shape[0]):
    wtv[i][0] = round(wtv[i][0])
    wtv[i][1] = round(wtv[i][1])
    wtv[i][2] = round(wtv[i][2])

In [ ]:
n_shonkha = 0
for i in range(wtv.shape[0]):
    if np.array_equal(wtv[i],new_ytest[i]):
        n_shonkha+=1
        
print(n_shonkha/wtv.shape[0])

In [ ]:
wtv

In [ ]:
new_ytest